In [8]:
import pandas as pd 
import numpy as np

specialEvents = pd.read_csv('../original_data/new_york_special_events.csv')

len(specialEvents)

13377

In [9]:
# Take out columns
specialEvents.drop(columns=['Organization Name','Facility Name','Direction','City','County','State','Event Description','Responding Organization Id'], inplace=True)

In [10]:
# Take out events that are not relevant

values_to_remove = ['Unkown Value', 'special events']
specialEvents = specialEvents[~specialEvents['Event Type'].isin(values_to_remove)]

In [11]:
# Parse out dates except all of 2012

specialEvents['Create Time'] = pd.to_datetime(specialEvents['Create Time'], format='%m/%d/%Y %I:%M:%S %p')
specialEvents['Close Time'] = pd.to_datetime(specialEvents['Close Time'], format='%m/%d/%Y %I:%M:%S %p')

start_date = '2012-01-01'
end_date = '2012-12-31'

specialEvents = specialEvents[(specialEvents['Create Time'] >= start_date) & (specialEvents['Close Time'] <= end_date)]

len(specialEvents)

0

In [12]:
# Parse out new york taxi coordinates

min_lat = 40.52763504199989
max_lat = 40.91037152011096
min_lon = -74.21220034099993
max_lon = -73.70134715908382

specialEvents = specialEvents[(specialEvents['Latitude'] >= min_lat) & (specialEvents['Latitude'] <= max_lat) & (specialEvents['Longitude'] >= min_lon) & (specialEvents['Longitude'] <= max_lon)]

len(specialEvents)

0

In [13]:
# Creating a new row for each hour for each event duration

def create_hourly_rows(row):

    hourly_range = pd.date_range(start=row['Create Time'], end=row['Close Time'], freq='H', closed='left')
    
    repeats = len(hourly_range)
    if repeats == 0:  
        hourly_range = [row['Create Time']]  
        repeats = 1
    new_rows = pd.DataFrame(np.repeat(row.values[None, :], repeats, axis=0), columns=row.index)
    new_rows['start_datetime'] = hourly_range
    new_rows['end_datetime'] = row['Close Time']

    new_rows['start_datetime'] = pd.to_datetime(new_rows['start_datetime']) 
    new_rows['end_datetime'] = pd.to_datetime(new_rows['end_datetime'])

    dif = (new_rows['end_datetime'] - new_rows['start_datetime']).dt.total_seconds() / 60

    new_rows.loc[dif >= 59, 'end_datetime'] = new_rows['start_datetime'] + pd.Timedelta(minutes=59)

    return new_rows

specialEvents = pd.concat(specialEvents.apply(create_hourly_rows, axis=1).tolist(), ignore_index=True)
    

C:\Users\hethe\AppData\Local\Temp\ipykernel_26408\3907797993.py:5: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  hourly_range = pd.date_range(start=row['Create Time'], end=row['Close Time'], freq='H', closed='left')


AttributeError: 'DataFrame' object has no attribute 'tolist'

In [ ]:
#specialEvents.rename(columns={'Create Time': ' start_datetime'}, inplace=True)
#specialEvents.rename(columns={'Close Time': ' end_datetime'}, inplace=True)
specialEvents.to_csv('../interim_refined_data/refined_ny_special_events.csv', index=False)